In [1]:
from dcodebeauty.datos import read_file
from dcodebeauty.pre_utils import find_description
from dcodebeauty.ocr import find_product_description,detect_text,cleaning
from gensim.models.ldamodel import LdaModel
from gensim_func import id2word,corpus

2022-07-02 18:52:47.879510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-02 18:52:47.879555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
ruta_modelo = '../models/model_3_topics'
model = LdaModel.load(ruta_modelo)
corpus = corpus

In [3]:
text = 'moisture bond refreshing hydra-gel oil free'
product = find_product_description(text)
clean_product = cleaning(product)
description = find_description(clean_product)

Found 3.51% of ingredients


In [8]:
len(description['descriptions'])

41

In [13]:
model[id2word.doc2bow(description['descriptions'][0].split())]


In [ ]:
for i in range(len(description)):
    

In [14]:
print(description['ingredients'][0],res1)

Butylene Glycol [(0, 0.03077827), (1, 0.89889514), (2, 0.070326574)]


In [20]:
chemic = (res1 [1])
chemic

(1, 0.89889514)